### Installer les bibliotèques nécessaires :

Pour PyTDC, nous n'installons pas les dépendances car tileDBsoma (l'une de ces dernières) a de gros problème de compatibilité (en tout cas sur ma machine)

Les dépendances requises pour PyTDC sont celles-ci : numpy, pandas, scipy et torch et seront installées indépendamment

Lien documentation TDC : https://tdc.readthedocs.io/en/main/install.html

In [38]:
%pip install transformers
%pip install setuptools
%pip install biopython
%pip install anndata
%pip install pandas
%pip install mygene
%pip install torch
%pip install scipy
%pip install numpy

%pip install PyTDC --no-deps

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### 1. Charger le dataset

In [39]:
import pandas as pd

# Charger le dataset
aml = pd.read_csv('../data processing/reduced_dataset/aml_anova.csv', sep=",", on_bad_lines="skip")

# Transposer les données et mettre en forme le DataFrame
data = aml.transpose()

data.columns = data.iloc[0]
data = data[1:]

data.reset_index(drop=True, inplace=True)

### 2. Préparation

2.1. Charger scGPT et son Tokenizer

In [40]:
import torch

from tdc import tdc_hf_interface
from tdc.model_server.tokenizers.scgpt import scGPTTokenizer

# Charger le modèle scGPT via TDC HuggingFace interface
scgpt = tdc_hf_interface("scGPT")
model = scgpt.load() 

# Charger le tokenizer spécifique pour scGPT
tokenizer = scGPTTokenizer()

2.2. Préparer les données aux bons formats

In [41]:
import numpy as np

# Récupérer les noms des gènes à partir du dataset
gene_names = data.columns[:-2] 

# Exclure les labels et ne conserver que les valeurs d'expression
data_values = data.iloc[:, :-2].values  

# Convertir les valeurs en float (en cas de type non numérique)
data_values = data_values.astype(np.float32)

# Remplacer les NaN (si présents) 
data_values = np.nan_to_num(data_values, nan=0.0)

# Vérifier les types des données avant de les tokeniser
print(f"Type des valeurs d'expression : {data_values.dtype}")
print(f"Type des noms des gènes : {type(gene_names)}")

# Convertir les noms des gènes en liste de chaînes si ce n'est pas déjà le cas
if not isinstance(gene_names, list):
    gene_names = gene_names.tolist()

# Ajouter le jeton '<cls>' au début de la liste des gènes
gene_names = ['<cls>'] + gene_names

# Ajouter une colonne vide (ou des zéros) à data_values pour faire correspondre le nombre de gènes
data_values = np.hstack([np.zeros((data_values.shape[0], 1), dtype=np.float32), data_values])

# Tokeniser les données en utilisant la méthode tokenize_cell_vectors
tokenized_data = tokenizer.tokenize_cell_vectors(data_values, np.array(gene_names))

Found local copy...


Type des valeurs d'expression : float32
Type des noms des gènes : <class 'pandas.core.indexes.base.Index'>


### 3. Préparation du chunking

In [ ]:
import torch
from torch.utils.data import DataLoader

chunk_sizes = [3, 5, 10, 20, 400]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}\n")
model.to(device)

def process_in_chunks(tokenized_data, chunk_size):

    dataloader = DataLoader(tokenized_data, batch_size=chunk_size, shuffle=False)
    row_embeddings = []

    for batch in dataloader:

        input_ids, gene_tokens = batch
        
        # Créer un masque d'attention (mask) pour les gènes (pas de padding ici)
        mask = torch.tensor(gene_tokens != 0, dtype=torch.bool).to(device)
        
        input_ids = input_ids.to(device)
        gene_tokens = gene_tokens.to(device)

        # Inférence par lots
        with torch.no_grad():

            outputs = model(input_ids, gene_tokens, attention_mask=mask)
            row_embeddings.append(outputs['cell_emb'].cpu().numpy())

    # Convertir les embeddings en un tableau NumPy
    return np.vstack(row_embeddings)


Using device: cuda



### 4. Embedding et exports

In [43]:
import os
from torch.nn.utils.rnn import pad_sequence

for chunk_size in chunk_sizes:

    print(f"Processing with chunk size: {chunk_size}")

    # Utiliser le DataLoader sans padding
    dataloader = DataLoader(tokenized_data, batch_size=chunk_size, shuffle=False)

    row_embeddings = []

    # Traiter chaque batch
    for batch in dataloader:

        input_ids, gene_tokens = batch
        print(f"Batch input_ids shape: {input_ids.shape}, gene_tokens shape: {gene_tokens.shape}")

        # Inférence par lots
        with torch.no_grad():
            outputs = model(input_ids, gene_tokens, attention_mask=(gene_tokens != 0).to(device))
            row_embeddings.append(outputs['cell_emb'])

# Padder des embeddings
padded_row_embeddings = pad_sequence(row_embeddings, batch_first=True, padding_value=0)

# Convertir les embeddings en un tableau NumPy
row_embeddings = padded_row_embeddings.cpu().numpy()
print(f"Embeddings shape après padding: {row_embeddings.shape}")

# Exporter les embeddings
output_dir = f"embeddings_chunk_{chunk_size}"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, f"scgpt_aml_embeddings_chunk_{chunk_size}.csv")

# Ajouter les labels aux embeddings
pd_embeddings = pd.DataFrame(row_embeddings)

embeddings_set = pd.concat([data.iloc[:, -2:], pd_embeddings], axis=1)
embeddings_set.to_csv(output_path, index=False, header=True)

# Vérifier les résultats
csv = pd.read_csv(output_path, sep=",", on_bad_lines="skip")

print(f"Shape des embeddings (chunk {chunk_size}):", row_embeddings.shape)
print(csv.head())

Processing with chunk size: 3
Batch input_ids shape: torch.Size([3, 158]), gene_tokens shape: torch.Size([3, 158])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

# END